In [12]:
#importing necesary packaged
import json
import pandas as pd

from ari import ari_to_features
import pandas as pd
from tqdm import tqdm
import re

### importing the first 1000 playlists in to a variable called json

In [13]:
#Loading the first json file, with the forst 1000 playlist into a variable
with open('/Users/jovitagandhi/Python - In class/Capstone 1 - Spotify recommendation system/mpd.slice.0-999.json') as f:
    data = json.load(f)

### Now flatenning it all into a dataframe
- To do this, we run a for aloop thorugh the entire json file, and use every playlist level information and song level information to the dataframe, in one row

In [14]:
#running a loop to save it all into one dataframe
# Create an empty list to store the flattened playlists
flattened_playlists = []

# Iterate over each playlist
for playlist in data['playlists']:

    # Iterate over each track in the playlist
    for track in playlist['tracks']:

        # Create a new dictionary to store the flattened data
        flattened_data = {}

        # Add the playlist-level data to the flattened dictionary
        flattened_data['pid'] = playlist['pid']
        flattened_data['name'] = playlist['name']
        flattened_data['collaborative'] = playlist['collaborative']
        flattened_data['modified_at'] = playlist['modified_at']
        flattened_data['num_tracks'] = playlist['num_tracks']
        flattened_data['num_albums'] = playlist['num_albums']
        flattened_data['num_followers'] = playlist['num_followers']

        # Add the track-level data to the flattened dictionary
        flattened_data['pos'] = track['pos']
        flattened_data['artist_name'] = track['artist_name']
        flattened_data['track_uri'] = track['track_uri']
        flattened_data['artist_uri'] = track['artist_uri']
        flattened_data['track_name'] = track['track_name']
        flattened_data['album_uri'] = track['album_uri']
        flattened_data['duration_ms'] = track['duration_ms']
        flattened_data['album_name'] = track['album_name']

        # Add the flattened data to the list
        flattened_playlists.append(flattened_data)

# Create a DataFrame from the flattened list of playlists
df = pd.DataFrame(flattened_playlists)

### now that we stored everything into one data frame, we check the shape, if there are any null values, and if there any duplicates

In [15]:
df.head()

,pid,name,collaborative,modified_at,num_tracks,num_albums,num_followers,pos,artist_name,track_uri,artist_uri,track_name,album_uri,duration_ms,album_name
0,0,Throwbacks,false,1493424000,52,47,1,0,Missy Elliott,spotify:track:0UaMYEvWZi0ZqiDOoHU3YI,spotify:artist:2wIVse2owClT7go1WT98tk,Lose Control (feat. Ciara & Fat Man Scoop),spotify:album:6vV5UrXcfyQD1wu4Qo2I9K,226863,The Cookbook
1,0,Throwbacks,false,1493424000,52,47,1,1,Britney Spears,spotify:track:6I9VzXrHxO9rA9A5euc8Ak,spotify:artist:26dSoYclwsYLMAKD3tpOr4,Toxic,spotify:album:0z7pVBGOD7HCIB7S8eLkLI,198800,In The Zone
2,0,Throwbacks,false,1493424000,52,47,1,2,Beyoncé,spotify:track:0WqIKmW4BTrj3eJFmnCKMv,spotify:artist:6vWDO969PvNqNYHIOW5v0m,Crazy In Love,spotify:album:25hVFAxTlDvXbx2X2QkUkE,235933,Dangerously In Love (Alben für die Ewigkeit)
3,0,Throwbacks,false,1493424000,52,47,1,3,Justin Timberlake,spotify:track:1AWQoqb9bSvzTjaLralEkT,spotify:artist:31TPClRtHm23RisEBtV3X7,Rock Your Body,spotify:album:6QPkyl04rXwTGlGlcYaRoW,267266,Justified
4,0,Throwbacks,false,1493424000,52,47,1,4,Shaggy,spotify:track:1lzr43nnXAijIGYnCT8M8H,spotify:artist:5EvFsr3kj42KNv97ZEnqij,It Wasn't Me,spotify:album:6NmFmPX56pcLBOFMhIiKvF,227600,Hot Shot


In [16]:
df.isnull().sum()

pid              0
name             0
collaborative    0
modified_at      0
num_tracks       0
num_albums       0
num_followers    0
pos              0
artist_name      0
track_uri        0
artist_uri       0
track_name       0
album_uri        0
duration_ms      0
album_name       0
dtype: int64

In [17]:
df.shape

(67503, 15)

In [18]:
df.dtypes

pid               int64
name             object
collaborative    object
modified_at       int64
num_tracks        int64
num_albums        int64
num_followers     int64
pos               int64
artist_name      object
track_uri        object
artist_uri       object
track_name       object
album_uri        object
duration_ms       int64
album_name       object
dtype: object

In [19]:
df.describe()

,pid,modified_at,num_tracks,num_albums,num_followers,pos,duration_ms
count,67503.000000,6.750300e+04,67503.000000,67503.000000,67503.000000,67503.000000,6.750300e+04
mean,508.371317,1.481839e+09,112.276447,81.050887,3.006829,55.638223,2.331839e+05
std,286.288365,3.476186e+07,63.771477,49.287440,35.920805,49.051327,6.455234e+04
min,0.000000,1.322611e+09,5.000000,2.000000,1.000000,0.000000,0.000000e+00
25%,262.000000,1.469232e+09,57.000000,40.500000,1.000000,17.000000,1.984130e+05
50%,505.000000,1.496448e+09,104.000000,73.000000,1.000000,41.000000,2.248660e+05
75%,752.000000,1.507075e+09,159.000000,115.000000,2.000000,82.000000,2.580400e+05
max,999.000000,1.509494e+09,245.000000,225.000000,1038.000000,244.000000,2.424189e+06


### Since we 67503 songs, we stick to using only 1 of the 100 json files provided, die to lack of computational power

#### - Using spotify's API, we extract information about each song. The fucntion to do this has been made in ari.py.
- To do this, we need to change the "track_uri"'s column into a readble format, so we remove the "spotify:track"

In [20]:
#first we extract 
df["track_uri"] = df["track_uri"].apply(lambda x: re.findall(r'\w+$', x)[0])
df["track_uri"]

0        0UaMYEvWZi0ZqiDOoHU3YI
1        6I9VzXrHxO9rA9A5euc8Ak
2        0WqIKmW4BTrj3eJFmnCKMv
3        1AWQoqb9bSvzTjaLralEkT
4        1lzr43nnXAijIGYnCT8M8H
                  ...          
67498    5uCax9HTNlzGybIStD3vDh
67499    0P1oO2gREMYUCoOkzYAyFu
67500    2oM4BuruDnEvk59IvIXCwn
67501    4Ri5TTUgjM96tbQZd5Ua7V
67502    5RVuBrXVLptAEbGJdSDzL5
Name: track_uri, Length: 67503, dtype: object

In [21]:
#as an example, we first test:
#Test the feature extraction script, and display features
ari_to_features(df["track_uri"][1275])

{'danceability': 0.576,
 'energy': 0.989,
 'key': 2,
 'loudness': -1.909,
 'mode': 1,
 'speechiness': 0.112,
 'acousticness': 0.00252,
 'instrumentalness': 0.0032,
 'liveness': 0.0243,
 'valence': 0.352,
 'tempo': 135.093,
 'type': 'audio_features',
 'id': '2MvIMgtWyK88OiPi0J8Dg3',
 'uri': 'spotify:track:2MvIMgtWyK88OiPi0J8Dg3',
 'track_href': 'https://api.spotify.com/v1/tracks/2MvIMgtWyK88OiPi0J8Dg3',
 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/2MvIMgtWyK88OiPi0J8Dg3',
 'duration_ms': 283400,
 'time_signature': 4,
 'artist_pop': 78,
 'genres': 'alternative_metal nu_metal rap_metal rock',
 'track_pop': 68}

### now that it's in a legible format, we can extract features about each song, add to this which features

#as an example, we first test:
#Test the feature extraction script, and display features
ari_to_features(df["track_uri"][0])

### We will run the code for each track to get these features, and do this by first splitting the data into three different sections,a dn store it back into a dataframe, we split it on the basis of the track_uri, because that's what we're looking for to use to add the features

In [13]:
first_half = df["track_uri"].unique()[:500]
#second_half = df["track_uri"].unique()[10000:20000]
#third_half = df["track_uri"].unique()[20000:]
dataLIST = [first_half]

In [12]:
import time

featureLIST = []

for i in tqdm([uri for uri in dataLIST[0]]):
    try:
        start_time = time.time()
        feature = ari_to_features(i)
        elapsed_time = time.time() - start_time
        
        if elapsed_time > 5:
            print(f"Skipping track {i}: elapsed time = {elapsed_time:.2f} seconds")
            continue
        
        featureLIST.append(feature)
    except:
        continue


100%|██████████| 500/500 [06:48<00:00,  1.22it/s]


## Now we actually run the code for each of the songs, and add all the features to each song and store it into a new dataframe, called feature_df

In [14]:
feature_df = pd.DataFrame(featureLIST)
feature_df

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,...,type,id,uri,track_href,analysis_url,duration_ms,time_signature,artist_pop,genres,track_pop
0,0.904,0.813,4,-7.105,0,0.1210,0.03110,0.006970,0.0471,0.810,...,audio_features,0UaMYEvWZi0ZqiDOoHU3YI,spotify:track:0UaMYEvWZi0ZqiDOoHU3YI,https://api.spotify.com/v1/tracks/0UaMYEvWZi0Z...,https://api.spotify.com/v1/audio-analysis/0UaM...,226864,4,73,dance_pop hip_hop hip_pop neo_soul pop_rap r&b...,69
1,0.774,0.838,5,-3.914,0,0.1140,0.02490,0.025000,0.2420,0.924,...,audio_features,6I9VzXrHxO9rA9A5euc8Ak,spotify:track:6I9VzXrHxO9rA9A5euc8Ak,https://api.spotify.com/v1/tracks/6I9VzXrHxO9r...,https://api.spotify.com/v1/audio-analysis/6I9V...,198800,4,81,dance_pop pop,83
2,0.664,0.758,2,-6.583,0,0.2100,0.00238,0.000000,0.0598,0.701,...,audio_features,0WqIKmW4BTrj3eJFmnCKMv,spotify:track:0WqIKmW4BTrj3eJFmnCKMv,https://api.spotify.com/v1/tracks/0WqIKmW4BTrj...,https://api.spotify.com/v1/audio-analysis/0WqI...,235933,4,86,pop r&b,21
3,0.892,0.714,4,-6.055,0,0.1410,0.20100,0.000234,0.0521,0.817,...,audio_features,1AWQoqb9bSvzTjaLralEkT,spotify:track:1AWQoqb9bSvzTjaLralEkT,https://api.spotify.com/v1/tracks/1AWQoqb9bSvz...,https://api.spotify.com/v1/audio-analysis/1AWQ...,267267,4,80,dance_pop pop,78
4,0.853,0.606,0,-4.596,1,0.0713,0.05610,0.000000,0.3130,0.654,...,audio_features,1lzr43nnXAijIGYnCT8M8H,spotify:track:1lzr43nnXAijIGYnCT8M8H,https://api.spotify.com/v1/tracks/1lzr43nnXAij...,https://api.spotify.com/v1/audio-analysis/1lzr...,227600,4,74,dance_pop pop_rap reggae_fusion,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,0.566,0.861,10,-7.543,1,0.0365,0.14300,0.000213,0.0817,0.862,...,audio_features,4s6Q9zPmiYcpo5MR7uoVnt,spotify:track:4s6Q9zPmiYcpo5MR7uoVnt,https://api.spotify.com/v1/tracks/4s6Q9zPmiYcp...,https://api.spotify.com/v1/audio-analysis/4s6Q...,155493,5,56,country country_road,56
496,0.696,0.449,7,-10.481,1,0.0292,0.55500,0.000000,0.2800,0.689,...,audio_features,58e5NT8x76aW7vfg2HxIpi,spotify:track:58e5NT8x76aW7vfg2HxIpi,https://api.spotify.com/v1/tracks/58e5NT8x76aW...,https://api.spotify.com/v1/audio-analysis/58e5...,181280,4,45,classic_country_pop outlaw_country,40
497,0.378,0.570,8,-12.024,1,0.1140,0.72300,0.000017,0.1510,0.851,...,audio_features,1r717VaFVWg0GpgMGwWssL,spotify:track:1r717VaFVWg0GpgMGwWssL,https://api.spotify.com/v1/tracks/1r717VaFVWg0...,https://api.spotify.com/v1/audio-analysis/1r71...,162533,4,52,bakersfield_sound,19
498,0.711,0.491,7,-10.323,1,0.0295,0.38100,0.000009,0.2490,0.671,...,audio_features,3otZLoLcx2d74hx8RaHXh6,spotify:track:3otZLoLcx2d74hx8RaHXh6,https://api.spotify.com/v1/tracks/3otZLoLcx2d7...,https://api.spotify.com/v1/audio-analysis/3otZ...,167933,4,59,contemporary_country country country_road,53


In [15]:
feature_df.shape

(500, 21)

## Finalising the feature data frame

We finally merge the feature DataFrame with the original dataset, as this also contains useful information in the artist name and track name. This is now our processed data

In [16]:
feature_df.columns

Index(['danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness',
       'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo',
       'type', 'id', 'uri', 'track_href', 'analysis_url', 'duration_ms',
       'time_signature', 'artist_pop', 'genres', 'track_pop'],
      dtype='object')

Creating a copy first

In [19]:
df2 = df[:500]

In [20]:
df2.shape

(500, 15)

##  Renamed the column in featureDF "uri" to "track_uri" so we can join them now

In [21]:
feature_df = feature_df.rename(columns={"uri": "track_uri"})


In [22]:
feature_df.columns

Index(['danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness',
       'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo',
       'type', 'id', 'track_uri', 'track_href', 'analysis_url', 'duration_ms',
       'time_signature', 'artist_pop', 'genres', 'track_pop'],
      dtype='object')

In [23]:
#first we extract 
feature_df["track_uri"] = feature_df["track_uri"].apply(lambda x: re.findall(r'\w+$', x)[0])
feature_df["track_uri"]

0      0UaMYEvWZi0ZqiDOoHU3YI
1      6I9VzXrHxO9rA9A5euc8Ak
2      0WqIKmW4BTrj3eJFmnCKMv
3      1AWQoqb9bSvzTjaLralEkT
4      1lzr43nnXAijIGYnCT8M8H
                ...          
495    4s6Q9zPmiYcpo5MR7uoVnt
496    58e5NT8x76aW7vfg2HxIpi
497    1r717VaFVWg0GpgMGwWssL
498    3otZLoLcx2d74hx8RaHXh6
499    5Afue9VTRQFArYOjtyUCri
Name: track_uri, Length: 500, dtype: object

In [24]:
feature_df.head()

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,...,type,id,track_uri,track_href,analysis_url,duration_ms,time_signature,artist_pop,genres,track_pop
0,0.904,0.813,4,-7.105,0,0.1210,0.03110,0.006970,0.0471,0.810,...,audio_features,0UaMYEvWZi0ZqiDOoHU3YI,0UaMYEvWZi0ZqiDOoHU3YI,https://api.spotify.com/v1/tracks/0UaMYEvWZi0Z...,https://api.spotify.com/v1/audio-analysis/0UaM...,226864,4,73,dance_pop hip_hop hip_pop neo_soul pop_rap r&b...,69
1,0.774,0.838,5,-3.914,0,0.1140,0.02490,0.025000,0.2420,0.924,...,audio_features,6I9VzXrHxO9rA9A5euc8Ak,6I9VzXrHxO9rA9A5euc8Ak,https://api.spotify.com/v1/tracks/6I9VzXrHxO9r...,https://api.spotify.com/v1/audio-analysis/6I9V...,198800,4,81,dance_pop pop,83
2,0.664,0.758,2,-6.583,0,0.2100,0.00238,0.000000,0.0598,0.701,...,audio_features,0WqIKmW4BTrj3eJFmnCKMv,0WqIKmW4BTrj3eJFmnCKMv,https://api.spotify.com/v1/tracks/0WqIKmW4BTrj...,https://api.spotify.com/v1/audio-analysis/0WqI...,235933,4,86,pop r&b,21
3,0.892,0.714,4,-6.055,0,0.1410,0.20100,0.000234,0.0521,0.817,...,audio_features,1AWQoqb9bSvzTjaLralEkT,1AWQoqb9bSvzTjaLralEkT,https://api.spotify.com/v1/tracks/1AWQoqb9bSvz...,https://api.spotify.com/v1/audio-analysis/1AWQ...,267267,4,80,dance_pop pop,78
4,0.853,0.606,0,-4.596,1,0.0713,0.05610,0.000000,0.3130,0.654,...,audio_features,1lzr43nnXAijIGYnCT8M8H,1lzr43nnXAijIGYnCT8M8H,https://api.spotify.com/v1/tracks/1lzr43nnXAij...,https://api.spotify.com/v1/audio-analysis/1lzr...,227600,4,74,dance_pop pop_rap reggae_fusion,0


## Merging the two

In [25]:
new_df = pd.merge(df2, feature_df, on='track_uri', how='left')

In [26]:
new_df.head()

,pid,name,collaborative,modified_at,num_tracks,num_albums,num_followers,pos,artist_name,track_uri,...,tempo,type,id,track_href,analysis_url,duration_ms_y,time_signature,artist_pop,genres,track_pop
0,0,Throwbacks,false,1493424000,52,47,1,0,Missy Elliott,0UaMYEvWZi0ZqiDOoHU3YI,...,125.461,audio_features,0UaMYEvWZi0ZqiDOoHU3YI,https://api.spotify.com/v1/tracks/0UaMYEvWZi0Z...,https://api.spotify.com/v1/audio-analysis/0UaM...,226864,4,73,dance_pop hip_hop hip_pop neo_soul pop_rap r&b...,69
1,0,Throwbacks,false,1493424000,52,47,1,1,Britney Spears,6I9VzXrHxO9rA9A5euc8Ak,...,143.040,audio_features,6I9VzXrHxO9rA9A5euc8Ak,https://api.spotify.com/v1/tracks/6I9VzXrHxO9r...,https://api.spotify.com/v1/audio-analysis/6I9V...,198800,4,81,dance_pop pop,83
2,0,Throwbacks,false,1493424000,52,47,1,2,Beyoncé,0WqIKmW4BTrj3eJFmnCKMv,...,99.259,audio_features,0WqIKmW4BTrj3eJFmnCKMv,https://api.spotify.com/v1/tracks/0WqIKmW4BTrj...,https://api.spotify.com/v1/audio-analysis/0WqI...,235933,4,86,pop r&b,21
3,0,Throwbacks,false,1493424000,52,47,1,3,Justin Timberlake,1AWQoqb9bSvzTjaLralEkT,...,100.972,audio_features,1AWQoqb9bSvzTjaLralEkT,https://api.spotify.com/v1/tracks/1AWQoqb9bSvz...,https://api.spotify.com/v1/audio-analysis/1AWQ...,267267,4,80,dance_pop pop,78
4,0,Throwbacks,false,1493424000,52,47,1,4,Shaggy,1lzr43nnXAijIGYnCT8M8H,...,94.759,audio_features,1lzr43nnXAijIGYnCT8M8H,https://api.spotify.com/v1/tracks/1lzr43nnXAij...,https://api.spotify.com/v1/audio-analysis/1lzr...,227600,4,74,dance_pop pop_rap reggae_fusion,0


In [27]:
new_df.columns

Index(['pid', 'name', 'collaborative', 'modified_at', 'num_tracks',
       'num_albums', 'num_followers', 'pos', 'artist_name', 'track_uri',
       'artist_uri', 'track_name', 'album_uri', 'duration_ms_x', 'album_name',
       'danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness',
       'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo',
       'type', 'id', 'track_href', 'analysis_url', 'duration_ms_y',
       'time_signature', 'artist_pop', 'genres', 'track_pop'],
      dtype='object')

## Now pre process for making the similarity matrix 

In [28]:
# Import library
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_extraction.text import TfidfVectorizer
from textblob import TextBlob
import re

### Preprocessing

The following cells conducts further preprocessing for the to cater specifically for the content-based filtering.

Here is the general pipeline:
1. Useful data Selection
2. List concatenation

#### Useful Data Selection

Due to the nature of playlist, there will be duplicates in songs across multiple playlists. Therefore, combined the song and the artist and used the `drop_duplicates()` function in `pandas` to remove duplicate songs when building the base dataframe with all unique songs.

In [74]:
#new_df[['artist_name','track_name','name']]

,artist_name,track_name,name
0,Missy Elliott,Lose Control (feat. Ciara & Fat Man Scoop),Throwbacks
1,Britney Spears,Toxic,Throwbacks
2,Beyoncé,Crazy In Love,Throwbacks
3,Justin Timberlake,Rock Your Body,Throwbacks
4,Shaggy,It Wasn't Me,Throwbacks
...,...,...,...
495,The Cadillac Three,Drunk Like You,old country
496,Chris Lane,Fix,old country
497,Travis Tritt,It's A Great Day To Be Alive,old country
498,Alan Jackson,Chattahoochee - Extended Mix,old country


In [29]:
new_df.drop_duplicates(subset=['track_uri'], keep='first', inplace=True)

In [30]:
# Drop song duplicates
def drop_duplicates(new_df):
    '''
    Drop duplicate songs
    '''
    new_df['artists_song'] = new_df.apply(lambda row: row['artist_name']+row['track_name'],axis = 1)
    return new_df.drop_duplicates('artists_song')

song_df = drop_duplicates(new_df)
print("Are all songs unique: ",len(pd.unique(song_df.artists_song))==len(song_df))


Are all songs unique:  True


In [31]:
new_df.shape

(489, 36)

## Pre processing by definig some functions 

In [32]:
# Select useful columns
def select_cols(new_df):
       '''
       Select useful columns
       '''
       return new_df[['artist_name','id','track_name','danceability', 'energy', 'key', 'loudness', 'mode',
       'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', "artist_pop", "genres", "track_pop"]]
song_df = select_cols(new_df)
song_df.head()

,artist_name,id,track_name,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,artist_pop,genres,track_pop
0,Missy Elliott,0UaMYEvWZi0ZqiDOoHU3YI,Lose Control (feat. Ciara & Fat Man Scoop),0.904,0.813,4,-7.105,0,0.1210,0.03110,0.006970,0.0471,0.810,125.461,73,dance_pop hip_hop hip_pop neo_soul pop_rap r&b...,69
1,Britney Spears,6I9VzXrHxO9rA9A5euc8Ak,Toxic,0.774,0.838,5,-3.914,0,0.1140,0.02490,0.025000,0.2420,0.924,143.040,81,dance_pop pop,83
2,Beyoncé,0WqIKmW4BTrj3eJFmnCKMv,Crazy In Love,0.664,0.758,2,-6.583,0,0.2100,0.00238,0.000000,0.0598,0.701,99.259,86,pop r&b,21
3,Justin Timberlake,1AWQoqb9bSvzTjaLralEkT,Rock Your Body,0.892,0.714,4,-6.055,0,0.1410,0.20100,0.000234,0.0521,0.817,100.972,80,dance_pop pop,78
4,Shaggy,1lzr43nnXAijIGYnCT8M8H,It Wasn't Me,0.853,0.606,0,-4.596,1,0.0713,0.05610,0.000000,0.3130,0.654,94.759,74,dance_pop pop_rap reggae_fusion,0


#### List Concatenation

After selecting the useful data, due to the import format of a dataframe, we need to convert the `genres` columns back into a list. This is done by using the `split()` function:

In [33]:
def genre_preprocess(song_df):
    '''
    Preprocess the genre data
    '''
    song_df['genres_list'] = song_df['genres'].apply(lambda x: x.split(" "))
    return song_df
song_df = genre_preprocess(song_df)
song_df['genres_list'].head()


/var/folders/q9/_jmy2jy533vdw8fbp9d85t5r0000gn/T/ipykernel_80865/2871997192.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  song_df['genres_list'] = song_df['genres'].apply(lambda x: x.split(" "))


0    [dance_pop, hip_hop, hip_pop, neo_soul, pop_ra...
1                                     [dance_pop, pop]
2                                           [pop, r&b]
3                                     [dance_pop, pop]
4                  [dance_pop, pop_rap, reggae_fusion]
Name: genres_list, dtype: object

## Creating a pipe line

In [86]:
# def playlist_preprocess(df3):
#     '''
#     Preprocess imported playlist
#     '''
#     df3= drop_duplicates(df3)
#     df3 = select_cols(df3)
#     df3 = genre_preprocess(df3)

#     return df3

### Feature Generation
Now that the data is usable, we can now feature-engineer the data for the purpose of the recommendation system. In this project, the following process is conducted into a pipeline for feature generation.

1. Sentiment Analysis
2. One-hot Encoding
3. TF-IDF
4. Normalization

#### Sentiment Analysis

In our data, we will perform a simply sentiment analysis using subjectivity and polarity form `TextBlob` package.
- **Subjectivity** (0,1): The amount of personal opinion and factual information contained in the text.
- **Polarity** (-1,1): The degree of strong or clearly defined sentiment accounting for negation.

We will then use one-hot encoding to list the sentiment of the song titles as one of the input.

In [34]:
def getSubjectivity(text):
  return TextBlob(text).sentiment.subjectivity

def getPolarity(text):
  return TextBlob(text).sentiment.polarity

def getAnalysis(score, task="polarity"):
  if task == "subjectivity":
    if score < 1/3:
      return "low"
    elif score > 1/3:
      return "high"
    else:
      return "medium"
  else:
    if score < 0:
      return 'Negative'
    elif score == 0:
      return 'Neutral'
    else:
      return 'Positive'

def sentiment_analysis(song_df, text_col):
  song_df['subjectivity'] = song_df[text_col].apply(getSubjectivity).apply(lambda x: getAnalysis(x,"subjectivity"))
  song_df['polarity'] = song_df[text_col].apply(getPolarity).apply(getAnalysis)
  return song_df

In [35]:
# Show result
sentiment = sentiment_analysis(song_df, "track_name")
sentiment.head()

/var/folders/q9/_jmy2jy533vdw8fbp9d85t5r0000gn/T/ipykernel_80865/3693717527.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  song_df['subjectivity'] = song_df[text_col].apply(getSubjectivity).apply(lambda x: getAnalysis(x,"subjectivity"))
/var/folders/q9/_jmy2jy533vdw8fbp9d85t5r0000gn/T/ipykernel_80865/3693717527.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  song_df['polarity'] = song_df[text_col].apply(getPolarity).apply(getAnalysis)


,artist_name,id,track_name,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,artist_pop,genres,track_pop,genres_list,subjectivity,polarity
0,Missy Elliott,0UaMYEvWZi0ZqiDOoHU3YI,Lose Control (feat. Ciara & Fat Man Scoop),0.904,0.813,4,-7.105,0,0.1210,0.03110,0.006970,0.0471,0.810,125.461,73,dance_pop hip_hop hip_pop neo_soul pop_rap r&b...,69,"[dance_pop, hip_hop, hip_pop, neo_soul, pop_ra...",low,Neutral
1,Britney Spears,6I9VzXrHxO9rA9A5euc8Ak,Toxic,0.774,0.838,5,-3.914,0,0.1140,0.02490,0.025000,0.2420,0.924,143.040,81,dance_pop pop,83,"[dance_pop, pop]",low,Neutral
2,Beyoncé,0WqIKmW4BTrj3eJFmnCKMv,Crazy In Love,0.664,0.758,2,-6.583,0,0.2100,0.00238,0.000000,0.0598,0.701,99.259,86,pop r&b,21,"[pop, r&b]",high,Negative
3,Justin Timberlake,1AWQoqb9bSvzTjaLralEkT,Rock Your Body,0.892,0.714,4,-6.055,0,0.1410,0.20100,0.000234,0.0521,0.817,100.972,80,dance_pop pop,78,"[dance_pop, pop]",low,Neutral
4,Shaggy,1lzr43nnXAijIGYnCT8M8H,It Wasn't Me,0.853,0.606,0,-4.596,1,0.0713,0.05610,0.000000,0.3130,0.654,94.759,74,dance_pop pop_rap reggae_fusion,0,"[dance_pop, pop_rap, reggae_fusion]",low,Neutral


In [36]:
def ohe_prep(song_df, column, new_name): 
    
    tf_df = pd.get_dummies(song_df[column])
    feature_names = tf_df.columns
    tf_df.columns = [new_name + "|" + str(i) for i in feature_names]
    tf_df.reset_index(drop = True, inplace = True)    
    return tf_df

In [37]:
subject_ohe = ohe_prep(sentiment, 'subjectivity','subject')
subject_ohe.iloc[0]

subject|high      0
subject|low       1
subject|medium    0
Name: 0, dtype: uint8

In [91]:
# TF-IDF implementation
tfidf = TfidfVectorizer()
tfidf_matrix =  tfidf.fit_transform(song_df['genres_list'].apply(lambda x: " ".join(x)))
genre_df = pd.DataFrame(tfidf_matrix.toarray())
genre_df.columns = ['genre' + "|" + i for i in tfidf.get_feature_names_out()]
#genre_df.drop(columns='genre|unknown')
genre_df.reset_index(drop = True, inplace=True)
genre_df.iloc[0]

genre|_indie              0.0
genre|acoustic_pop        0.0
genre|adult_standards     0.0
genre|aesthetic_rap       0.0
genre|afrofuturism        0.0
                         ... 
genre|welsh_rock          0.0
genre|west_coast_rap      0.0
genre|world_devotional    0.0
genre|yacht_rock          0.0
genre|zolo                0.0
Name: 0, Length: 346, dtype: float64

In [41]:
# artist_pop distribution descriptive stats
print(song_df['artist_pop'].describe())

count    489.000000
mean      61.282209
std       18.441259
min        0.000000
25%       50.000000
50%       64.000000
75%       74.000000
max       91.000000
Name: artist_pop, dtype: float64


#### Normalization
Lastly, we need to normalize some variables. As shown below, the popularity variables are not normalized to 0 to 1, which would be problematic in the consine similarity function later on. In addition, the audio features are also not normalized. 

WE thus use the MinMax scaler to normalize

In [42]:
pop = song_df[["artist_pop"]].reset_index(drop = True)
scaler = MinMaxScaler()
pop_scaled = pd.DataFrame(scaler.fit_transform(pop), columns = pop.columns)
pop_scaled.head()

,artist_pop
0,0.802198
1,0.890110
2,0.945055
3,0.879121
4,0.813187


In [44]:
song_df.head()

,artist_name,id,track_name,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,artist_pop,genres,track_pop,genres_list,subjectivity,polarity
0,Missy Elliott,0UaMYEvWZi0ZqiDOoHU3YI,Lose Control (feat. Ciara & Fat Man Scoop),0.904,0.813,4,-7.105,0,0.1210,0.03110,0.006970,0.0471,0.810,125.461,73,dance_pop hip_hop hip_pop neo_soul pop_rap r&b...,69,"[dance_pop, hip_hop, hip_pop, neo_soul, pop_ra...",low,Neutral
1,Britney Spears,6I9VzXrHxO9rA9A5euc8Ak,Toxic,0.774,0.838,5,-3.914,0,0.1140,0.02490,0.025000,0.2420,0.924,143.040,81,dance_pop pop,83,"[dance_pop, pop]",low,Neutral
2,Beyoncé,0WqIKmW4BTrj3eJFmnCKMv,Crazy In Love,0.664,0.758,2,-6.583,0,0.2100,0.00238,0.000000,0.0598,0.701,99.259,86,pop r&b,21,"[pop, r&b]",high,Negative
3,Justin Timberlake,1AWQoqb9bSvzTjaLralEkT,Rock Your Body,0.892,0.714,4,-6.055,0,0.1410,0.20100,0.000234,0.0521,0.817,100.972,80,dance_pop pop,78,"[dance_pop, pop]",low,Neutral
4,Shaggy,1lzr43nnXAijIGYnCT8M8H,It Wasn't Me,0.853,0.606,0,-4.596,1,0.0713,0.05610,0.000000,0.3130,0.654,94.759,74,dance_pop pop_rap reggae_fusion,0,"[dance_pop, pop_rap, reggae_fusion]",low,Neutral


In [46]:
def create_feature_set(song_df, float_cols):
    
    # Tfidf genre lists
    tfidf = TfidfVectorizer()
    tfidf_matrix =  tfidf.fit_transform(song_df['genres_list'].apply(lambda x: " ".join(x)))
    genre_df = pd.DataFrame(tfidf_matrix.toarray())
    genre_df.columns = ['genre' + "|" + i for i in tfidf.get_feature_names_out()]
    genre_df.drop(columns='genre|unknown') # drop unknown genre
    genre_df.reset_index(drop = True, inplace=True)
    
    # Sentiment analysis
    song_df = sentiment_analysis(song_df, "track_name")

    # One-hot Encoding
    subject_ohe = ohe_prep(song_df, 'subjectivity','subject') * 0.3
    polar_ohe = ohe_prep(song_df, 'polarity','polar') * 0.5
    key_ohe = ohe_prep(song_df, 'key','key') * 0.5
    mode_ohe = ohe_prep(song_df, 'mode','mode') * 0.5

    # Normalization
    # Scale popularity columns
    pop = song_df[["artist_pop","track_pop"]].reset_index(drop = True)
    scaler = MinMaxScaler()
    pop_scaled = pd.DataFrame(scaler.fit_transform(pop), columns = pop.columns) * 0.2 

    # Scale audio columns
    floats = song_df[float_cols].reset_index(drop = True)
    scaler = MinMaxScaler()
    floats_scaled = pd.DataFrame(scaler.fit_transform(floats), columns = floats.columns) * 0.2

    # Concanenate all features
    final = pd.concat([genre_df, floats_scaled, pop_scaled, subject_ohe, polar_ohe, key_ohe, mode_ohe], axis = 1)
    
    # Add song id
    final['id']=song_df['id'].values
    
    return final

In [47]:
# Save the data and generate the features
float_cols = song_df.dtypes[song_df.dtypes == 'float64'].index.values

# Generate features
complete_feature_set = create_feature_set(song_df, float_cols=float_cols)
complete_feature_set.head()


/var/folders/q9/_jmy2jy533vdw8fbp9d85t5r0000gn/T/ipykernel_80865/3693717527.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  song_df['subjectivity'] = song_df[text_col].apply(getSubjectivity).apply(lambda x: getAnalysis(x,"subjectivity"))
/var/folders/q9/_jmy2jy533vdw8fbp9d85t5r0000gn/T/ipykernel_80865/3693717527.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  song_df['polarity'] = song_df[text_col].apply(getPolarity).apply(getAnalysis)


,genre|_indie,genre|acoustic_pop,genre|adult_standards,genre|aesthetic_rap,genre|afrofuturism,genre|alabama_indie,genre|album_rock,genre|alt_z,genre|alternative_dance,genre|alternative_hip_hop,...,key|5,key|6,key|7,key|8,key|9,key|10,key|11,mode|0,mode|1,id
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.0,0UaMYEvWZi0ZqiDOoHU3YI
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.0,6I9VzXrHxO9rA9A5euc8Ak
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.0,0WqIKmW4BTrj3eJFmnCKMv
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.0,1AWQoqb9bSvzTjaLralEkT
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,1lzr43nnXAijIGYnCT8M8H


### Content-based Filtering Recommendation
The next step is to perform content-based filtering based on the song features we have. To do so, we concatenate all songs in a playlist into one summarization vector. Then, we find the similarity between the summarized playlist vector with all songs (not including the songs in the playlist) in the database. Then, we use the similarity measure retrieved the most relevant song that is not in the playlist to recommend it.

There are thre steps in this section:
1. **Choose playlist**: In this part, we retrieve a playlist
2. **Extract features**: In this part, we retireve playlist-of-interest features and non-playlist-of-interest features.
3. **Find similarity**: In this part, we compare the summarized playlist features with all other songs.

#### Choose Playlist
In this part, we test using a playlist from my own spotify app, that i have created


In [48]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

CLIENT_ID = 'f7687f8c2b8c4be88dbaca006d7e8fe4'
CLIENT_SECRET = '81996f23979644c190bdb1be41b60428'
playlist_link = "https://open.spotify.com/playlist/41SAIGuQpUAqA5zQpaTJoL?si=909498b3f30245f2"

CLIENT_CREDENTIALS_MANAGER = SpotifyClientCredentials(
    client_id=CLIENT_ID, client_secret=CLIENT_SECRET
)
SP = spotipy.Spotify(client_credentials_manager=CLIENT_CREDENTIALS_MANAGER)


def get_playlist_uri(playlist_link):
    return playlist_link.split("/")[-1].split("?")[0]


def get_tracks():
    tracks_uri = []
    playlist_uri = get_playlist_uri(playlist_link)
    for track in SP.playlist_tracks(playlist_uri)["items"]:
        track_uri = track["track"]["uri"]
        track_name = track["track"]["name"]
        result = track_uri, SP.audio_features(track_uri)
        tracks_uri.append(result)

    return tracks_uri


In [49]:
get_playlist_uri(playlist_link)

'41SAIGuQpUAqA5zQpaTJoL'

In [50]:
tracks = get_tracks()

In [51]:
type(tracks)

list

In [122]:
tracks

[('spotify:track:4lD7gHKF1e8Y9stQuVezZB',
  [{'danceability': 0.396,
    'energy': 0.358,
    'key': 9,
    'loudness': -8.294,
    'mode': 1,
    'speechiness': 0.0292,
    'acousticness': 0.422,
    'instrumentalness': 1.71e-05,
    'liveness': 0.19,
    'valence': 0.111,
    'tempo': 103.807,
    'type': 'audio_features',
    'id': '4lD7gHKF1e8Y9stQuVezZB',
    'uri': 'spotify:track:4lD7gHKF1e8Y9stQuVezZB',
    'track_href': 'https://api.spotify.com/v1/tracks/4lD7gHKF1e8Y9stQuVezZB',
    'analysis_url': 'https://api.spotify.com/v1/audio-analysis/4lD7gHKF1e8Y9stQuVezZB',
    'duration_ms': 271467,
    'time_signature': 4}]),
 ('spotify:track:6hkOqJ5mE093AQf2lbZnsG',
  [{'danceability': 0.691,
    'energy': 0.795,
    'key': 1,
    'loudness': -4.021,
    'mode': 0,
    'speechiness': 0.0355,
    'acousticness': 0.000812,
    'instrumentalness': 1.02e-06,
    'liveness': 0.156,
    'valence': 0.876,
    'tempo': 133.971,
    'type': 'audio_features',
    'id': '6hkOqJ5mE093AQf2lbZnsG'

In [52]:
## Flattening to a dataframe:

import pandas as pd



# Create an empty list to store the flattened tracks
flattened_tracks = []

# Iterate over each tuple in the list
for uri, audio_features in tracks:

    # Create a new dictionary to store the flattened data
    flattened_track = {}

    # Add the track-level data to the flattened dictionary
    flattened_track['uri'] = uri
    for key, value in audio_features[0].items():
        flattened_track[key] = value

    # Add the flattened data to the list
    flattened_tracks.append(flattened_track)

# Create a DataFrame from the flattened list of tracks
df_test = pd.DataFrame(flattened_tracks)




In [53]:
df_test.head()

,uri,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,track_href,analysis_url,duration_ms,time_signature
0,spotify:track:4lD7gHKF1e8Y9stQuVezZB,0.396,0.358,9,-8.294,1,0.0292,0.422000,0.000017,0.190,0.111,103.807,audio_features,4lD7gHKF1e8Y9stQuVezZB,https://api.spotify.com/v1/tracks/4lD7gHKF1e8Y...,https://api.spotify.com/v1/audio-analysis/4lD7...,271467,4
1,spotify:track:6hkOqJ5mE093AQf2lbZnsG,0.691,0.795,1,-4.021,0,0.0355,0.000812,0.000001,0.156,0.876,133.971,audio_features,6hkOqJ5mE093AQf2lbZnsG,https://api.spotify.com/v1/tracks/6hkOqJ5mE093...,https://api.spotify.com/v1/audio-analysis/6hkO...,227333,4
2,spotify:track:0sY6ZUTh4yoctD8VIXz339,0.462,0.240,0,-12.077,1,0.0377,0.920000,0.000000,0.113,0.320,171.319,audio_features,0sY6ZUTh4yoctD8VIXz339,https://api.spotify.com/v1/tracks/0sY6ZUTh4yoc...,https://api.spotify.com/v1/audio-analysis/0sY6...,244000,4
3,spotify:track:4Dvkj6JhhA12EX05fT7y2e,0.520,0.731,6,-5.338,0,0.0557,0.342000,0.001010,0.311,0.662,173.930,audio_features,4Dvkj6JhhA12EX05fT7y2e,https://api.spotify.com/v1/tracks/4Dvkj6JhhA12...,https://api.spotify.com/v1/audio-analysis/4Dvk...,167303,4
4,spotify:track:7kt9e9LFSpN1zQtYEl19o1,0.515,0.476,9,-9.843,1,0.0328,0.314000,0.000047,0.171,0.412,135.886,audio_features,7kt9e9LFSpN1zQtYEl19o1,https://api.spotify.com/v1/tracks/7kt9e9LFSpN1...,https://api.spotify.com/v1/audio-analysis/7kt9...,195097,4


In [55]:
### This is the test data    

def select_cols(df_test):
       '''
       Select useful columns
       '''
       return new_df[['id','track_name','danceability', 'energy', 'key', 'loudness', 'mode',
       'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', "artist_pop", "genres", "track_pop"]]
df_test = select_cols(df_test)
df_test.head()


,id,track_name,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,artist_pop,genres,track_pop
0,0UaMYEvWZi0ZqiDOoHU3YI,Lose Control (feat. Ciara & Fat Man Scoop),0.904,0.813,4,-7.105,0,0.1210,0.03110,0.006970,0.0471,0.810,125.461,73,dance_pop hip_hop hip_pop neo_soul pop_rap r&b...,69
1,6I9VzXrHxO9rA9A5euc8Ak,Toxic,0.774,0.838,5,-3.914,0,0.1140,0.02490,0.025000,0.2420,0.924,143.040,81,dance_pop pop,83
2,0WqIKmW4BTrj3eJFmnCKMv,Crazy In Love,0.664,0.758,2,-6.583,0,0.2100,0.00238,0.000000,0.0598,0.701,99.259,86,pop r&b,21
3,1AWQoqb9bSvzTjaLralEkT,Rock Your Body,0.892,0.714,4,-6.055,0,0.1410,0.20100,0.000234,0.0521,0.817,100.972,80,dance_pop pop,78
4,1lzr43nnXAijIGYnCT8M8H,It Wasn't Me,0.853,0.606,0,-4.596,1,0.0713,0.05610,0.000000,0.3130,0.654,94.759,74,dance_pop pop_rap reggae_fusion,0


In [56]:
def genre_preprocess(df_test):
    df_test['genres_list'] = df_test['genres'].apply(lambda x: x.split(" "))
    return song_df
df_test = genre_preprocess(df_test)
df_test['genres_list'].head()

/var/folders/q9/_jmy2jy533vdw8fbp9d85t5r0000gn/T/ipykernel_80865/3718756168.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test['genres_list'] = df_test['genres'].apply(lambda x: x.split(" "))


0    [dance_pop, hip_hop, hip_pop, neo_soul, pop_ra...
1                                     [dance_pop, pop]
2                                           [pop, r&b]
3                                     [dance_pop, pop]
4                  [dance_pop, pop_rap, reggae_fusion]
Name: genres_list, dtype: object

In [57]:
df_test.head()

,artist_name,id,track_name,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,artist_pop,genres,track_pop,genres_list,subjectivity,polarity
0,Missy Elliott,0UaMYEvWZi0ZqiDOoHU3YI,Lose Control (feat. Ciara & Fat Man Scoop),0.904,0.813,4,-7.105,0,0.1210,0.03110,0.006970,0.0471,0.810,125.461,73,dance_pop hip_hop hip_pop neo_soul pop_rap r&b...,69,"[dance_pop, hip_hop, hip_pop, neo_soul, pop_ra...",low,Neutral
1,Britney Spears,6I9VzXrHxO9rA9A5euc8Ak,Toxic,0.774,0.838,5,-3.914,0,0.1140,0.02490,0.025000,0.2420,0.924,143.040,81,dance_pop pop,83,"[dance_pop, pop]",low,Neutral
2,Beyoncé,0WqIKmW4BTrj3eJFmnCKMv,Crazy In Love,0.664,0.758,2,-6.583,0,0.2100,0.00238,0.000000,0.0598,0.701,99.259,86,pop r&b,21,"[pop, r&b]",high,Negative
3,Justin Timberlake,1AWQoqb9bSvzTjaLralEkT,Rock Your Body,0.892,0.714,4,-6.055,0,0.1410,0.20100,0.000234,0.0521,0.817,100.972,80,dance_pop pop,78,"[dance_pop, pop]",low,Neutral
4,Shaggy,1lzr43nnXAijIGYnCT8M8H,It Wasn't Me,0.853,0.606,0,-4.596,1,0.0713,0.05610,0.000000,0.3130,0.654,94.759,74,dance_pop pop_rap reggae_fusion,0,"[dance_pop, pop_rap, reggae_fusion]",low,Neutral


In [58]:
playlist_info = create_feature_set(df_test,float_cols)

/var/folders/q9/_jmy2jy533vdw8fbp9d85t5r0000gn/T/ipykernel_80865/3693717527.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  song_df['subjectivity'] = song_df[text_col].apply(getSubjectivity).apply(lambda x: getAnalysis(x,"subjectivity"))
/var/folders/q9/_jmy2jy533vdw8fbp9d85t5r0000gn/T/ipykernel_80865/3693717527.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  song_df['polarity'] = song_df[text_col].apply(getPolarity).apply(getAnalysis)


#### Find similarity
The last puzzle is to find the similarities between the summarized playlist vector and all other songs. There are many similarity measures but one of the most common measures is **cosine similarity**.

Cosine similarity is a mathematical value that measures the similarities between vectors. Imagining our songs vectors as only two dimensional, the visual representation would look similar to the figure below. 

The mathematical formula can be expressed as:
$$\text{Cosine Sim}(A,B)=\frac{A\cdot B}{||A||\times||B||}=\frac{\sum_{i=1}^n A_i\times B_i}{\sqrt{\sum_{i=1}^n A_i^2}\times \sqrt{\sum_{i=1}^n B_i^2}}$$

In our code, we used the `cosine_similarity()` function from `scikit learn` to measure the similarity between each song and the summarized playlist vector.

One big advatange of doing this is the time complexity of the whole algorithm is equal to a matrix multiplication since we are performing the cosine similarity measure between each row vector (song) and the column vector of summarized playlist feature.

In [59]:
complete_feature_set.columns

Index(['genre|_indie', 'genre|acoustic_pop', 'genre|adult_standards',
       'genre|aesthetic_rap', 'genre|afrofuturism', 'genre|alabama_indie',
       'genre|album_rock', 'genre|alt_z', 'genre|alternative_dance',
       'genre|alternative_hip_hop',
       ...
       'key|5', 'key|6', 'key|7', 'key|8', 'key|9', 'key|10', 'key|11',
       'mode|0', 'mode|1', 'id'],
      dtype='object', length=378)

In [60]:
final_set = complete_feature_set = complete_feature_set.drop('id', axis=1)


In [61]:
final_test_set = playlist_info.drop('id', axis=1)

In [62]:

cosine_similarities = cosine_similarity(final_set, final_test_set)

In [63]:
# Sort the similarity values in descending order
song_indices = cosine_similarities.argsort()[0][::-1]

# Select the top N songs as recommendations
N = 489
top_song_indices = song_indices[:N]

In [64]:
song_df['top_indices'] = top_song_indices.tolist()

/var/folders/q9/_jmy2jy533vdw8fbp9d85t5r0000gn/T/ipykernel_80865/345817049.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  song_df['top_indices'] = top_song_indices.tolist()


In [65]:
top_songs = song_df.iloc[top_song_indices][['artist_name', 'track_name']]

In [68]:
top_songs.sort_index(ascending= True, inplace = True)

In [69]:
top_songs[:10]

,artist_name,track_name
0,Missy Elliott,Lose Control (feat. Ciara & Fat Man Scoop)
1,Britney Spears,Toxic
2,Beyoncé,Crazy In Love
3,Justin Timberlake,Rock Your Body
4,Shaggy,It Wasn't Me
5,Usher,Yeah!
6,Usher,My Boo
7,The Pussycat Dolls,Buttons
8,Destiny's Child,Say My Name
9,OutKast,Hey Ya! - Radio Mix / Club Mix
